In [1]:
import pandas as pd
from utils import DataDic
from utils import LogParser
import constants as config
import copy
import numpy as np
import json

In [3]:
# 11.12.2021 full dataset version
def create_user_table():
    user_dic = {}
    f = open('raw_dataset/20211112user.json')
    user_data = json.load(f)
    
    for user_record in user_data['data']:
        #print(user_record)
        image_sequence = user_record['userImagesV2']
        user_id = user_record['userID']
        user_dic[user_id] = image_sequence
    #print(user_dic)
    # apply to the user.csv
    user_pd = pd.read_csv('raw_dataset/users.csv', encoding = 'utf-8')
    count = 0
    for i in range(len(user_pd)):
        user_name = user_pd.at[i,'M_username']
        if user_name in user_dic:
            user_pd.at[i,'M_assignment_by_image_id'] = user_dic[user_name]
            #print(user_name, len(user_dic[user_name].split('; ')))
        count += len(user_dic[user_name].split('; '))
    user_pd.to_csv('raw_dataset/user_1121.csv', index=False, encoding='utf_8_sig')

def create_annotation_table():
    #DataDic(config.V1_DATA_PATH,'ImageID', data_format='csv')
    annotation_dic = DataDic('raw_dataset/20211112annotation.csv','imageID', data_format='csv', key2='userID')
    #annotation_dic.get_value('I6674Y2000-MS')
    data = pd.read_csv('raw_dataset/annotations.csv', encoding = 'utf-8')
    for i in range(len(data)): #len(data)
        image_id = data.at[i,'image_id']
        username = data.at[i,'username']
        key = image_id + '-' + username
        record = annotation_dic.get_value(key)

        user_log = record['history_logs']
        user_log_date = record['history_list_date']
        user_log, user_log_date = LogParser.preprocess_log(user_log, user_log_date)
        user_log, user_log_date = LogParser.log_rules(username + ' ' + image_id, user_log, user_log_date)
        user_log, user_log_date = LogParser.log_parser_1121(user_log, user_log_date, username)
        # print(user_log, user_log_date)
        data.at[i,'annotation_log'] = ';'.join(user_log)
        data.at[i,'log_dates'] = ';'.join(user_log_date)
        data.at[i,'is_error_image'] = int(record['isMark'])
        data.at[i,'need_discuss'] = int(record['needAnotherCoder'])
        data.at[i,'marked_fun'] = int(record['isFunFigure'])
        data.at[i,'marked_OK'] = int(record['isHappyFigure'])
        for label_id in config.docanno2visanno:
            tag = config.docanno2visanno[label_id]
            if(tag in record):
                data.loc[i,label_id] = record[tag]
            else:
                if(label_id == 'lab_25'):
                    if(record['ifNested'] == 0):
                        data.at[i,label_id] = 'lab_26'
                    elif(record['ifNested'] == 1):
                        data.at[i,label_id] = 'lab_27'
                    elif(record['ifNested'] == 2):
                        data.at[i,label_id] = 'lab_28'
                elif(label_id == 'lab_29'):
                    if(record['hardness'] == 1):
                        data.at[i,label_id] = 'lab_30'
                    elif(record['hardness'] == 2):
                        data.at[i,label_id] = 'lab_31'
                    elif(record['hardness'] == 3):
                        data.at[i,label_id] = 'lab_32'
            
    data.to_csv('raw_dataset/annotations_1121.csv', index=False, encoding='utf_8_sig')

create_annotation_table()

create_user_table()

In [7]:
# 11.12.2021 filtered version, only keep images with single label choosen from encoding list
def create_user_table():
    filtered_image_pd = pd.read_csv('raw_dataset/20211112.filtered.csv', encoding = 'utf-8')
    filtered_images = list(filtered_image_pd['ImageID'])
    
    user_pd = pd.read_csv('raw_dataset/user_1121.csv', encoding = 'utf-8')
    count = 0
    for i in range(len(user_pd)):
        user_name = user_pd.at[i,'M_username']
        assignments = user_pd.at[i,'M_assignment_by_image_id'].split('; ')
        filtered_assignments = []
        for image in assignments:
            if image in filtered_images:
                filtered_assignments.append(image)
        count += len(filtered_assignments)
        #print(user_name, len(filtered_assignments))
        user_pd.at[i,'M_assignment_by_image_id'] = '; '.join(filtered_assignments)
    #print(count/2)
    user_pd.to_csv('raw_dataset/user_1121_filtered.csv', index=False, encoding='utf_8_sig')

#create_user_table()

def create_annotation_table():
    #DataDic(config.V1_DATA_PATH,'ImageID', data_format='csv')
    annotation_dic = DataDic('raw_dataset/20211112annotation.csv','imageID', data_format='csv', key2='userID')
    #annotation_dic.get_value('I6674Y2000-MS')
    data = pd.read_csv('raw_dataset/annotations_1121_filtered.csv', encoding = 'utf-8')
    for i in range(len(data)): #len(data)
        image_id = data.at[i,'image_id']
        username = data.at[i,'username']
        key = image_id + '-' + username
        record = annotation_dic.get_value(key)

        user_log = record['history_logs']
        user_log_date = record['history_list_date']
        user_log, user_log_date = LogParser.preprocess_log(user_log, user_log_date)
        user_log, user_log_date = LogParser.log_rules(username + ' ' + image_id, user_log, user_log_date)
        user_log, user_log_date = LogParser.log_parser_1121(user_log, user_log_date, username)
        # print(user_log, user_log_date)
        data.at[i,'annotation_log'] = ';'.join(user_log)
        data.at[i,'log_dates'] = ';'.join(user_log_date)
        data.at[i,'is_error_image'] = int(record['isMark'])
        data.at[i,'need_discuss'] = int(record['needAnotherCoder'])
        data.at[i,'marked_fun'] = int(record['isFunFigure'])
        data.at[i,'marked_OK'] = int(record['isHappyFigure'])
        for label_id in config.docanno2visanno:
            tag = config.docanno2visanno[label_id]
            if(tag in record):
                data.loc[i,label_id] = record[tag]
            else:
                if(label_id == 'lab_25'):
                    if(record['ifNested'] == 0):
                        data.at[i,label_id] = 'lab_26'
                    elif(record['ifNested'] == 1):
                        data.at[i,label_id] = 'lab_27'
                    elif(record['ifNested'] == 2):
                        data.at[i,label_id] = 'lab_28'
                elif(label_id == 'lab_29'):
                    if(record['hardness'] == 1):
                        data.at[i,label_id] = 'lab_30'
                    elif(record['hardness'] == 2):
                        data.at[i,label_id] = 'lab_31'
                    elif(record['hardness'] == 3):
                        data.at[i,label_id] = 'lab_32'
            
    data.to_csv('raw_dataset/annotations_1121_filtered_updated.csv', index=False, encoding='utf_8_sig')

create_annotation_table()

